In [90]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import optimizers
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import time

In [91]:
data = pd.read_csv('C:/900/news_train_from2013.csv')
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9328750 entries, 0 to 9328749
Data columns (total 36 columns):
Unnamed: 0              int64
time                    object
sourceTimestamp         object
firstCreated            object
sourceId                object
headline                object
urgency                 int64
takeSequence            int64
provider                object
subjects                object
audiences               object
bodySize                int64
companyCount            int64
headlineTag             object
marketCommentary        bool
sentenceCount           int64
wordCount               int64
assetCodes              object
assetName               object
firstMentionSentence    int64
relevance               float64
sentimentClass          int64
sentimentNegative       float64
sentimentNeutral        float64
sentimentPositive       float64
sentimentWordCount      int64
noveltyCount12H         int64
noveltyCount24H         int64
noveltyCount3D          int64

In [100]:
data = pd.DataFrame(data, columns = ['time','headline','urgency','assetName','relevance','sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive'])
data.head()

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
0,2007-01-01 04:29:32,China's Daqing pumps 43.41 mln tonnes of oil i...,3,PetroChina Co Ltd,0.235702,-1,0.500739,0.419327,0.079934
1,2007-01-01 07:03:35,"FEATURE-In kidnapping, finesse works best",3,Travelers Companies Inc,0.447214,-1,0.600082,0.345853,0.054064
2,2007-01-01 11:29:56,PRESS DIGEST - Wall Street Journal - Jan 1,3,Wal-Mart Stores Inc,0.377964,-1,0.450049,0.295671,0.254280
3,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,Google Inc,0.149071,-1,0.752917,0.162715,0.084368
4,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,XM Satellite Radio Holdings Inc,0.149071,-1,0.699274,0.209360,0.091367


In [93]:
import datetime
data['time'] = pd.to_datetime(data['time'], format = '%Y-%m-%d %H:%M:%S+00:00')

In [94]:
data.head()

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
0,2007-01-01 04:29:32,China's Daqing pumps 43.41 mln tonnes of oil i...,3,PetroChina Co Ltd,0.235702,-1,0.500739,0.419327,0.079934
1,2007-01-01 07:03:35,"FEATURE-In kidnapping, finesse works best",3,Travelers Companies Inc,0.447214,-1,0.600082,0.345853,0.054064
2,2007-01-01 11:29:56,PRESS DIGEST - Wall Street Journal - Jan 1,3,Wal-Mart Stores Inc,0.377964,-1,0.450049,0.295671,0.254280
3,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,Google Inc,0.149071,-1,0.752917,0.162715,0.084368
4,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,XM Satellite Radio Holdings Inc,0.149071,-1,0.699274,0.209360,0.091367


In [95]:
df=data.sort_values(by='time',
                    ascending=True)
df.head()

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
0,2007-01-01 04:29:32,China's Daqing pumps 43.41 mln tonnes of oil i...,3,PetroChina Co Ltd,0.235702,-1,0.500739,0.419327,0.079934
1,2007-01-01 07:03:35,"FEATURE-In kidnapping, finesse works best",3,Travelers Companies Inc,0.447214,-1,0.600082,0.345853,0.054064
2,2007-01-01 11:29:56,PRESS DIGEST - Wall Street Journal - Jan 1,3,Wal-Mart Stores Inc,0.377964,-1,0.450049,0.295671,0.254280
7,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,Microsoft Corp,0.447214,0,0.325929,0.374173,0.299898
6,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,Walt Disney Co,0.149071,0,0.152658,0.615878,0.231464


In [96]:
total=df.shape[0] 
endtime = df.loc[total-1,'time']
total

9328750

In [97]:
df2 = df.drop_duplicates('headline', keep='first', inplace=False)


In [98]:
df2.head()

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
0,2007-01-01 04:29:32,China's Daqing pumps 43.41 mln tonnes of oil i...,3,PetroChina Co Ltd,0.235702,-1,0.500739,0.419327,0.079934
1,2007-01-01 07:03:35,"FEATURE-In kidnapping, finesse works best",3,Travelers Companies Inc,0.447214,-1,0.600082,0.345853,0.054064
2,2007-01-01 11:29:56,PRESS DIGEST - Wall Street Journal - Jan 1,3,Wal-Mart Stores Inc,0.377964,-1,0.450049,0.295671,0.254280
7,2007-01-01 12:08:37,PRESS DIGEST - New York Times - Jan 1,3,Microsoft Corp,0.447214,0,0.325929,0.374173,0.299898
9,2007-01-01 13:00:02,Tenet Completes Sale of Alvarado Hospital Medi...,3,Tenet Healthcare Corp,1.000000,1,0.058059,0.139615,0.802326


In [99]:
total=df2.shape[0] 
total

5532379

In [283]:
CTXS= df2.loc[df2['assetName'].str.contains('Citrix Systems')]
CTXS

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
420,2007-01-02 15:00:02,How Small Businesses with Equally Small Budget...,3,Citrix Systems Inc,1.0000,1,0.056162,0.138773,0.805065
8712,2007-01-08 13:00:26,Citrix Completes Acquisition of Ardence Inc. B...,3,Citrix Systems Inc,1.0000,1,0.071154,0.146570,0.782276
11967,2007-01-09 13:45:02,Citrix Systems to Announce Fourth Quarter & Fu...,3,Citrix Systems Inc,1.0000,0,0.368470,0.512426,0.119103
14900,2007-01-10 14:00:31,Citrix Access Gateway Positioned in SSL VPN Le...,3,Citrix Systems Inc,1.0000,1,0.173993,0.171220,0.654787
23319,2007-01-16 15:00:25,New GoToMyPC 6.0 Embraces Sound to Seamlessly ...,3,Citrix Systems Inc,1.0000,1,0.236531,0.342520,0.420949
37527,2007-01-23 21:05:02,CITRIX REPORTS FOURTH QUARTER AND FISCAL YEAR ...,1,Citrix Systems Inc,1.0000,0,0.144968,0.617299,0.237734
37529,2007-01-23 21:05:05,CITRIX SYSTEMS INC <CTXS.O> Q4 SHR $0.32,1,Citrix Systems Inc,1.0000,0,0.014148,0.810077,0.175775
37552,2007-01-23 21:05:29,CITRIX SYSTEMS INC <CTXS.O> REUTERS ESTIMATES ...,1,Citrix Systems Inc,1.0000,0,0.016156,0.806295,0.177549
37556,2007-01-23 21:05:35,CITRIX SYSTEMS INC <CTXS.O> SEES Q1 NET REVENU...,1,Citrix Systems Inc,1.0000,0,0.163736,0.613386,0.222878
37558,2007-01-23 21:05:47,Citrix Reports Fourth Quarter and Fiscal Year ...,3,Citrix Systems Inc,1.0000,1,0.283357,0.345339,0.371304


In [165]:
INTC=INTC[ ~ INTC['assetName'].str.contains('Intellipharmaceutics')]
INTC

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
1461,2007-01-03 12:54:26,UPDATE 1-RESEARCH ALERT-Goldman Sachs downgrad...,3,Intel Corp,0.176777,-1,0.813457,0.127352,0.059191
4905,2007-01-04 16:00:08,Intel and CinemaNow Advance Burn-to-DVD Entert...,3,Intel Corp,1.000000,1,0.039804,0.126411,0.833785
5139,2007-01-04 17:59:17,"US STOCKS-Intel leads tech stocks higher, Dow ...",3,Intel Corp,1.000000,1,0.340777,0.170733,0.488490
5233,2007-01-04 19:00:29,"US STOCKS-Tech shares rise on Intel, oil hits Dow",3,Intel Corp,1.000000,1,0.114500,0.159669,0.725831
5466,2007-01-04 21:14:12,US STOCKS-Indexes end higher on technology's a...,3,Intel Corp,0.333333,0,0.025719,0.547729,0.426552
5628,2007-01-04 21:44:47,"US STOCKS-Oil's drop boosts stocks, Intel lead...",3,Intel Corp,1.000000,1,0.056160,0.137507,0.806333
5777,2007-01-04 22:44:57,"US STOCKS-Indexes end up as Intel lifts techs,...",3,Intel Corp,1.000000,1,0.058719,0.138951,0.802330
6399,2007-01-05 12:47:35,RESEARCH ALERT-CS reinstates Intel - theflyont...,3,Intel Corp,1.000000,0,0.016574,0.916779,0.066647
7591,2007-01-06 16:00:02,ADDING MULTIMEDIA Intel's Multi-Core Entertain...,3,Intel Corp,1.000000,1,0.013422,0.347659,0.638918
9477,2007-01-08 15:27:25,RESEARCH ALERT -Lehman cuts Advanced Micro Dev...,3,Intel Corp,0.192450,-1,0.780757,0.147288,0.071955


In [166]:
INTC=INTC[ ~ INTC['assetName'].str.contains('Inteliquent')]
INTC

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
1461,2007-01-03 12:54:26,UPDATE 1-RESEARCH ALERT-Goldman Sachs downgrad...,3,Intel Corp,0.176777,-1,0.813457,0.127352,0.059191
4905,2007-01-04 16:00:08,Intel and CinemaNow Advance Burn-to-DVD Entert...,3,Intel Corp,1.000000,1,0.039804,0.126411,0.833785
5139,2007-01-04 17:59:17,"US STOCKS-Intel leads tech stocks higher, Dow ...",3,Intel Corp,1.000000,1,0.340777,0.170733,0.488490
5233,2007-01-04 19:00:29,"US STOCKS-Tech shares rise on Intel, oil hits Dow",3,Intel Corp,1.000000,1,0.114500,0.159669,0.725831
5466,2007-01-04 21:14:12,US STOCKS-Indexes end higher on technology's a...,3,Intel Corp,0.333333,0,0.025719,0.547729,0.426552
5628,2007-01-04 21:44:47,"US STOCKS-Oil's drop boosts stocks, Intel lead...",3,Intel Corp,1.000000,1,0.056160,0.137507,0.806333
5777,2007-01-04 22:44:57,"US STOCKS-Indexes end up as Intel lifts techs,...",3,Intel Corp,1.000000,1,0.058719,0.138951,0.802330
6399,2007-01-05 12:47:35,RESEARCH ALERT-CS reinstates Intel - theflyont...,3,Intel Corp,1.000000,0,0.016574,0.916779,0.066647
7591,2007-01-06 16:00:02,ADDING MULTIMEDIA Intel's Multi-Core Entertain...,3,Intel Corp,1.000000,1,0.013422,0.347659,0.638918
9477,2007-01-08 15:27:25,RESEARCH ALERT -Lehman cuts Advanced Micro Dev...,3,Intel Corp,0.192450,-1,0.780757,0.147288,0.071955


In [167]:
INTC=INTC[ ~ INTC['assetName'].str.contains('Interactive')]
INTC

,time,headline,urgency,assetName,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive
1461,2007-01-03 12:54:26,UPDATE 1-RESEARCH ALERT-Goldman Sachs downgrad...,3,Intel Corp,0.176777,-1,0.813457,0.127352,0.059191
4905,2007-01-04 16:00:08,Intel and CinemaNow Advance Burn-to-DVD Entert...,3,Intel Corp,1.000000,1,0.039804,0.126411,0.833785
5139,2007-01-04 17:59:17,"US STOCKS-Intel leads tech stocks higher, Dow ...",3,Intel Corp,1.000000,1,0.340777,0.170733,0.488490
5233,2007-01-04 19:00:29,"US STOCKS-Tech shares rise on Intel, oil hits Dow",3,Intel Corp,1.000000,1,0.114500,0.159669,0.725831
5466,2007-01-04 21:14:12,US STOCKS-Indexes end higher on technology's a...,3,Intel Corp,0.333333,0,0.025719,0.547729,0.426552
5628,2007-01-04 21:44:47,"US STOCKS-Oil's drop boosts stocks, Intel lead...",3,Intel Corp,1.000000,1,0.056160,0.137507,0.806333
5777,2007-01-04 22:44:57,"US STOCKS-Indexes end up as Intel lifts techs,...",3,Intel Corp,1.000000,1,0.058719,0.138951,0.802330
6399,2007-01-05 12:47:35,RESEARCH ALERT-CS reinstates Intel - theflyont...,3,Intel Corp,1.000000,0,0.016574,0.916779,0.066647
7591,2007-01-06 16:00:02,ADDING MULTIMEDIA Intel's Multi-Core Entertain...,3,Intel Corp,1.000000,1,0.013422,0.347659,0.638918
9477,2007-01-08 15:27:25,RESEARCH ALERT -Lehman cuts Advanced Micro Dev...,3,Intel Corp,0.192450,-1,0.780757,0.147288,0.071955


In [284]:
CTXS.to_csv("CTXS.csv", encoding='utf-8')

In [285]:
data = pd.read_csv('CTXS.csv')
data.head

<bound method NDFrame.head of       Unnamed: 0                 time  \
0            420  2007-01-02 15:00:02   
1           8712  2007-01-08 13:00:26   
2          11967  2007-01-09 13:45:02   
3          14900  2007-01-10 14:00:31   
4          23319  2007-01-16 15:00:25   
5          37527  2007-01-23 21:05:02   
6          37529  2007-01-23 21:05:05   
7          37552  2007-01-23 21:05:29   
8          37556  2007-01-23 21:05:35   
9          37558  2007-01-23 21:05:47   
10         37566  2007-01-23 21:06:05   
11         37575  2007-01-23 21:06:23   
12         37576  2007-01-23 21:06:25   
13         37585  2007-01-23 21:06:50   
14         37589  2007-01-23 21:07:09   
15         37590  2007-01-23 21:07:25   
16         37598  2007-01-23 21:07:50   
17         37600  2007-01-23 21:07:52   
18         37602  2007-01-23 21:08:01   
19         37697  2007-01-23 21:18:43   
20         37706  2007-01-23 21:19:40   
21         37918  2007-01-23 21:57:24   
22         49918  2007-01-2

In [286]:
import datetime
df = data[['Unnamed: 0','time','headline', 'assetName']]
df['time'] = pd.to_datetime(data['time'], format = '%Y-%m-%d')
df

C:\Users\Jiang\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,time,headline,assetName
0,420,2007-01-02 15:00:02,How Small Businesses with Equally Small Budget...,Citrix Systems Inc
1,8712,2007-01-08 13:00:26,Citrix Completes Acquisition of Ardence Inc. B...,Citrix Systems Inc
2,11967,2007-01-09 13:45:02,Citrix Systems to Announce Fourth Quarter & Fu...,Citrix Systems Inc
3,14900,2007-01-10 14:00:31,Citrix Access Gateway Positioned in SSL VPN Le...,Citrix Systems Inc
4,23319,2007-01-16 15:00:25,New GoToMyPC 6.0 Embraces Sound to Seamlessly ...,Citrix Systems Inc
5,37527,2007-01-23 21:05:02,CITRIX REPORTS FOURTH QUARTER AND FISCAL YEAR ...,Citrix Systems Inc
6,37529,2007-01-23 21:05:05,CITRIX SYSTEMS INC <CTXS.O> Q4 SHR $0.32,Citrix Systems Inc
7,37552,2007-01-23 21:05:29,CITRIX SYSTEMS INC <CTXS.O> REUTERS ESTIMATES ...,Citrix Systems Inc
8,37556,2007-01-23 21:05:35,CITRIX SYSTEMS INC <CTXS.O> SEES Q1 NET REVENU...,Citrix Systems Inc
9,37558,2007-01-23 21:05:47,Citrix Reports Fourth Quarter and Fiscal Year ...,Citrix Systems Inc


In [287]:
df['Year'] = df['time'].dt.year
df['Month'] = df['time'].dt.month
df

C:\Users\Jiang\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Jiang\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Unnamed: 0,time,headline,assetName,Year,Month
0,420,2007-01-02 15:00:02,How Small Businesses with Equally Small Budget...,Citrix Systems Inc,2007,1
1,8712,2007-01-08 13:00:26,Citrix Completes Acquisition of Ardence Inc. B...,Citrix Systems Inc,2007,1
2,11967,2007-01-09 13:45:02,Citrix Systems to Announce Fourth Quarter & Fu...,Citrix Systems Inc,2007,1
3,14900,2007-01-10 14:00:31,Citrix Access Gateway Positioned in SSL VPN Le...,Citrix Systems Inc,2007,1
4,23319,2007-01-16 15:00:25,New GoToMyPC 6.0 Embraces Sound to Seamlessly ...,Citrix Systems Inc,2007,1
5,37527,2007-01-23 21:05:02,CITRIX REPORTS FOURTH QUARTER AND FISCAL YEAR ...,Citrix Systems Inc,2007,1
6,37529,2007-01-23 21:05:05,CITRIX SYSTEMS INC <CTXS.O> Q4 SHR $0.32,Citrix Systems Inc,2007,1
7,37552,2007-01-23 21:05:29,CITRIX SYSTEMS INC <CTXS.O> REUTERS ESTIMATES ...,Citrix Systems Inc,2007,1
8,37556,2007-01-23 21:05:35,CITRIX SYSTEMS INC <CTXS.O> SEES Q1 NET REVENU...,Citrix Systems Inc,2007,1
9,37558,2007-01-23 21:05:47,Citrix Reports Fourth Quarter and Fiscal Year ...,Citrix Systems Inc,2007,1


In [288]:
data1 = df.groupby(by=['Year','Month'])['headline'].sum()
data1.to_frame()

headline
Year Month                                                   
2007 1      How Small Businesses with Equally Small Budget...
     2      Citrix to Present at Upcoming Investor Confere...
     3      Citrix to Outline Business Strategy at Annual ...
     4      Florida Guardian ad Litem Office Uses Citrix A...
     5      Citrix Systems Positioned in Leaders Quadrant ...
     6      Citrix Broadens EdgeSight Product Line and Pro...
     7      Customers Celebrate Independence with Simpler,...
     8      Citrix Executives to Present at Upcoming Inves...
     9      U.S. approves Citrix purchase of XensourceCitr...
     10     Citrix Systems to Announce Third Quarter 2007 ...
     11     Citrix Executives to Present at Investor Confe...
     12     VMware says competition is not eroding pricing...
2008 1      Citrix Online Develops New Remote Support Solu...
     2      Citrix GoToMeeting Expands Reach and Shortens ...
     3      Parents Who Web Commute Balance Work and Famil...
     4      Citrix Unveils Groundbreaking New XenServer Pr...
     5      Citrix Synergy Attracts Tech Experts to the Ul...
     6      Citrix Online Chosen as Finalist for American ...
     7      Citrix Expands Channel Incentive Program for P...
     8      Citrix and Industry Leaders Rally Government f...
     9      US RESEARCH SUMMARY-S&P 500 Sept 15 1032 GMTCi...
     10     UPDATE 1-Goldman downgrades NetSuite, CommVaul...
     11     With Spotlight on Corporate Bailouts, Web Conf...
     12     Citrix to Host Second Delivery Center Live! Ev...
2009 1      Citrix Celebrates Presidential Inauguration wi...
     2      Citrix Branch Repeater Accelerates Delivery of...
     3      Citrix Simplifies Partner Program to Enable Ch...
     4      CITRIX JOINS DMTF BOARD OF DIRECTORSCitrix Sys...
     5      Citrix to Outline Business Strategy at Citrix ...
     6      Citrix Survey Finds Virtualization Top of Mind...
...                                                       ...
2014 7      Conference Call Schedules, Return of Popular S...
     8      Citrix Predicts the Future of Work <CTXS.O>Cit...
     9      Senior-level Appointments, Technical Updates, ...
     10     Citrix Named a Leader in Enterprise Mobile Man...
     11     Citrix Integrates GoToMeeting Free with Popula...
     12     CITRIX APPOINTS GEIR RAMLETH AS SENIOR VICE PR...
2015 1      CITRIX SYSTEMS <CTXS.O> - ON JAN 7, ENTERED IN...
     2      CORRECTING and REPLACING Citrix Executives to ...
     3      Citrix Introduces Next-Generation Carrier-Grad...
     4      Citrix Delivers a New, Intuitive User Experien...
     5      Citrix Outlines Vision for IT Transformation <...
     6      SAGE <SGE.L>: BERENBERG RAISES PRICE TARGET TO...
     7      Citrix Systems to Announce Second Quarter 2015...
     8      National Survey Finds Nearly 90% of People Use...
     9      CITRIX SYSTEMS INC SHARES DOWN 2.28% AT $66.56...
     10     The National Investor Relations Institute -- S...
     11     CITRIX SYSTEMS TO ANNOUNCE OPERATIONAL AND STR...
     12     Citrix Cited as a Leader in Enterprise Mobilit...
2016 1      CITRIX ACQUIRES COMTRADE'S SCOM MANAGEMENT PAC...
     2      CITRIX APPOINTS KEVIN PARKER AS CHAIRMAN OF TH...
     3      Citrix Radically Simplifies the Secure Deliver...
     4      RES APPOINTS NEW SVP OF GLOBAL SALES AND SERVI...
     5      Citrix to Outline Business Strategy at Synergy...
     6      Citrix GoTo Products Earn Prestigious Industry...
     7      Citrix Systems to Announce Second Quarter 2016...
     8      Citrix Signs Global Partner Agreement with Clo...
     9      Hokuhoku Financial Group, Inc. Adopts Citrix X...
     10     Citrix Systems to Announce Third Quarter 2016 ...
     11     Enhanced GoTo Product Portfolio Drives Better ...
     12     CITRIX SYSTEMS INC SHARES UP 1.8 PCT AT $87.12...

[120 rows x 1 columns]

In [289]:
data1.to_csv("CTXS_h.csv", encoding='utf-8')

In [19]:
data2 = data1[(data1['Year'] = '2007') & (data1['Month'] = '1')]#data chose from 2015-08 t0 2020-07
data2

SyntaxError: invalid syntax (<ipython-input-19-b75f56f63301>, line 1)

In [1]:
import pandas as pd
import numpy as np
data = pd.DataFrame({'id':[1,1,1,2,2,2],'value':['A','B','C','D','E','F']})
data1 = data.groupby(by='id')['value'].sum()

In [2]:
data1

id
1    ABC
2    DEF
Name: value, dtype: object

In [89]:
data = pd.read_csv('AMZN_h.csv')
data

,2007,1,"US RESEARCH NEWS-Citigroup raises Marsh & McLennan to buyUS STOCKS-Futures point higher on Wal-Mart, oil's declineUS STOCKS-Wall Street set to start new year higherAmazon launches shoe and handbag Web siteThree Lucky Libraries to Get the Gift of Literature Through Amazon.com's ''Wish for... <AMZN.O>Amazon.com to Webcast Fourth Quarter 2006 Financial Results Conference Call <AMZN.O>AMAZON.COM <AMZN.O> SHARES RISE 1.4 PCT IN ELECTRONIC TRADE AFTER BROKERAGE UPGRADE Before the Bell - Amazon.com shares riseAmazon.com's New Shoe and Handbag Website, Endless.com, Offers Customers Overnight... <AMZN.O>"
0,2007,2,Amazon.com Offers ''Harry Potter and the Death...
1,2007,3,Amazon.com to Webcast Investor Conference Pres...
2,2007,4,Amazon.com Pays Tribute to America's Pastime w...
3,2007,5,INTERVIEW-Microsoft eyes more Web infrastructu...
4,2007,6,PRESS DIGEST - Wall Street Journal - June 5RES...
5,2007,7,Amazon Announces Plans to Open New Fulfillment...
6,2007,8,CustomFlix Labs Changes Its Name to CreateSpac...
7,2007,9,NBC Universal Television Content Now Available...
8,2007,10,Writers compete for online readers for book de...
9,2007,11,SmartPay preparing for China's first e-payment...
